In [ ]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

# dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

In [ ]:
def split_array(f_split = 2.4e6, f_width_single_side = 4.e6, n_per_side = 4):
    f_center = 74.8e6
    # array1 = np.linspace( f_center-f_split/2-f_width_single_side ,f_center-f_split/2,n_per_side)
    array1 = f_center - f_split/2 - np.linspace( f_width_single_side , 0. ,n_per_side)
    # array2 = np.linspace( f_center+f_split/2+f_width_single_side ,f_center+f_split/2,n_per_side)
    array2 = f_center + f_split/2 + np.linspace( 0. , f_width_single_side ,n_per_side)
    # array1 = np.linspace(70.5e6,72.5e6,n)
    # array2 = np.linspace(77.5e6,79.5e6,n)
    return np.concatenate([array1,array2])

def compute_tweezer_1064_phases(n_tweezers,amplitudes):
    phase_tweezer_array = np.empty([n_tweezers])
    for tweezer_idx in range(n_tweezers):
        if tweezer_idx == 0:
            phase_tweezer_array[0] = 360
        else:
            phase_ij = 0
            for j in range(1,tweezer_idx):
                phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*amplitudes[tweezer_idx]
            phase_i = (phase_ij % 2*np.pi) * 360
            phase_tweezer_array[tweezer_idx] = phase_i
    return phase_tweezer_array

# f_list = split_array(f_split=2.4e6)
f_list = [71.2e6,76.e6,80.e6]
# f_list = [73.6e6,76.e6]

n_tweezers = len(f_list)

# a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
# a_list = [.1,.1,.1,.05,.1,.35]
a_list = [.54,.26,.19]

phases = compute_tweezer_1064_phases(len(f_list),a_list)

def normalize_alist(alist):
    sum = np.sum(alist)
    return a_list/sum
a_list = normalize_alist(a_list)

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
        dds[tweezer_idx].phase(phases[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()
# trigger.force()

# Start command including enable of trigger engine

# input("Press Enter to Exit")

In [ ]:
# fm the tweezer array for some amount of time, while simultaneously ramping down the modulation depth

# how much time to fm?
fm_time = 5.

# modulation frequency and depth
modulation_freq  = 1.
modulation_depth = 2.e6

# dummy variables to reflect experimental params
initial_optical_power = 5.
final_optical_power = 1.
final_over_initial_optical_power = initial_optical_power / final_optical_power

# rate at which to update awg frequency output ("sample" rate)
period_s = 20.e-3

# number of samples in a period
num_samples = int(1/(period_s * modulation_freq))

# total number of samples
total_samples = fm_time / period_s

# list of sample numbers (in ms)
sample_range = np.arange(total_samples)*period_s

# empty array to be filled with frequency values for each dds core
freq_lists = np.empty([len(f_list),len(sample_range)])

# compute and fill in the above array
for i in range(len(f_list)):
    freq_lists[i] = f_list[i] + modulation_depth * (1 - sample_range*(1 - final_over_initial_optical_power)/(total_samples*period_s))**(1/3) * np.sin(2*np.pi*sample_range*modulation_freq)

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(period_s)
dds.exec_at_trg()
dds.write_to_card()

# pre_fill the buffer
fill_max = dds.queue_cmd_max()
counter = 0
for counter in range(len(sample_range)):
    for tone_idx in range(np.shape(freq_lists)[0]):
        freq_Hz = freq_lists[tone_idx][counter]
        dds[tone_idx].freq(freq_Hz)
    dds.exec_at_trg()
dds.write_to_card()

# reset trigger mode to external at the end of the fm time
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()
